# DESCRIPTIVA PROCESO DE MACHINE LEARNING.
Nuestro objetivo es entender qué variables afectan a la máxima capacidad de producción de la máquina y cómo estas variables afectan a dicha capacidad.

PASO 1. OBTENCIÓN DE DATOS DESDE EL SCADA.
Para esta operación emplearemos la librería OpenOPC de Python 2.7 y elaboraremos 2 scripts y un fichero csv con las variables disponibles en el sistema vía conexión Server OPC. Así habremos obtenido nuestro dataset. "TBM_data.csv", el cual lo dividiremos en:
    - Datos de la primera mitad del año para el proceso de entrenamiento
    - Datos para el posterior análisis de estado, con 48 tuplas de datos.
Para ello emplearemos el código presente en el notebook "Obtencion_Fichero.ipynb" y el script get_files.py

PASO 2. PROCESO MACHINE LEARNING
El siguiente paso oes realizar un entrenamiento del sistema con los datos de la primera mitad del año, obtendremos un modelo matemático y unos estadísticos que nos permitirán valorar la validez del modelo. A tal efecto utilizaremos la libreria ciéntifica de Python Scipy y un script ModReg.py, que contiene una serie de funciones para la extracción, tratamiento y carga de datos, que permitan ser coherentes con los algoritmos de machine learning.

PASO 3. SELECCIÓN DE VARIABLES.

La selección de variables la realizaremos mediante dos algoritmos selectKBest y los mejores coeficientes de correlación obtenidos de las regresions lineales múltiples.

En este Notebook desarrollamos el paso 2. Al final del mismo mostramos los comentarios.

In [1]:
######### INPUTS DEL SISTEMA  ##############
file_data = 'data_train.csv' # Datos primera mitad del año.
var_remove = 'var_remove_TBM.csv' # variables descartadas 
#inicialmente bajo criterios de tipo de dato, y conocimientos 
#del proceso indiustrial 
criteria = 20 # Numero de variables elegidas para 
#la descripción del modelo

In [2]:
from ModReg_TG import imp_ds, Fit_Var, sense_var, f_model_reg, new_model
from f_selection import ForwardSelection_r2, ForwardSelection_r2_Ac, selectKvar, var_remove_DL

# Cargamos el dataset con todas las variables de la máquina 
#objetivo de estudio
file=imp_ds(file_data)
# devuelve lista de variable y el dataframe completo
    
#Función que elimina las variables no numéricas y establece 
#la variable dependiente
df_var=Fit_Var(file,var_remove)
# devuelve lista de variables tratadas, la X y la Y para el modelo
# de entrenamiento
    
#Obtención de los primeros estadísticos.
#Ejecutamos una función que devuelve el valor medio 
#y la desviación std
stat=sense_var(df_var[1],df_var[2])
# devuelve lista de variables y lista con las medias y 
#lista con las descv std.
    
#Una vez que el dataset ha sido tratado, se lanza la función 
#de regresión para el Modelo
#Se lanza tantas veces como sea necesario para asegurar que 
#no haya sobreajuste
e=0.1
i=0
while (e >0.05 and i <10):
    reg=f_model_reg(df_var[1],df_var[2])
    e=reg[2]
    if i >10:
        print('Modelo Presenta Sobre ajuste.')
    i+=1
print('------------------\nIteraciones realizadas >>>',i)
# devuelve una lista con los coeficientes de la regresion, 
# el bias de la regresion y la diferencia entre el r2 
#del train y el test

************************************************************
Tamaño DataSet: 3814 filas de datos y  168 variables registradas
************************************************************
R2 en training: 0.9910
R2 en test: 0.9820
Sobre Ajuste--->>> 0.0091
Error caudratico medio: 2174.84
Error absoluto medio: 35.94
Mediana del error Absoluto: 30.08
------------------
Iteraciones realizadas >>> 1


En este punto obtenemos que el entrenamiento nos devuelve un modelo con un r2 = 0.9908 y sin sobre ajuste. Parece un modelo razonable.

Además obtenemos dos fichero csv, con la siguiente información:
- Una lista con todas las variables o características del sistema. "file_var.csv"
- El modelo matematico, con los valores de los coeficientes y bias del modelo. "file_model.csv"


In [3]:
# Proceso de seleción de caracteristicas por orden de r2.
var_r2= ForwardSelection_r2_Ac(list(df_var[1].columns),
                               df_var[1],df_var[2])

In [4]:
var_r2[0:criteria]

,Variables,r2
0,Main manifold gas fuel pressure,0.934715
1,Compressor Inlet air temperature (T1),0.958987
2,Turbine temperature (T5) #12,0.973896
3,Presion diferencial primera etapa filtro aire,0.976168
4,Turbine Hours,0.979449
5,Inlet air filter differential pressure,0.983690
6,Compensated T5 average,0.985311
7,Pilot manifold gas fuel pressure,0.986740
8,Turbine temperature (T5) #9,0.987655
9,Turbine temperature (T5) #7,0.988450


In [5]:
# Valoramos modelo r2.
nm=new_model(list(var_r2[0:criteria]['Variables']),df_var[1],df_var[2])

# Obtenemos en fichero .csv la lista de variables a descartar en el 
# proceso de análisis
var_remove_DL(file_data,list(var_r2[0:criteria]['Variables']))
# Obtenemos un fichero con el modelo matemático de las variables 
# seleccionadas.

R2 en training: 0.9904
R2 en test: 0.9908
Sobre Ajuste--->>> 0.0004
Error caudratico medio: 2316.39
Error absoluto medio: 37.78
Mediana del error Absoluto: 31.63


Hemos obtenido un modelo con un r2=0.9908 y sin sobre ajuste con 20 variables, basadas en el algoritmo del mejor r2 acumulado y el fichero file_model.csv con el modelo matemático, que nos permitirá dadas unas condiciones en el proceso predecir el comportamiento de la máquina, así como, y lo más importante detectar desvíos de este modelo y causas, lo cual nos podrñá llevar a plantear soluciones y evitar pédidas de producción.

Además obtenemos el fichero var_remove_DL_XXXXX.csv, con la lista de variables a eliminar, y que no han sido seleccionadas en el algoritmo de selección, para los siguientes estudios análisis de estado de máquina.

# CONCLUSIONES:
Con 5 variables en el algoritmo de selección de variables del r2 acumulado, somos capaces de obtener el mismo resultado que con el de SekectKBest, tomando 20 variables, por lo que entendemos óptimo elegir este modelo, para predecir futuros comportamientos y realizsr análisis de estadod de la máquina.